In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Bert_Lime/IMDB/

/content/drive/MyDrive/Bert_Lime/IMDB


In [3]:
!pip install transformers
!pip install seqeval datasets allennlp
!pip install flax
!pip install sentencepiece
!pip install nltk
!pip install gensim

  Created wheel for jsonnet: filename=jsonnet-0.18.0-cp37-cp37m-linux_x86_64.whl size=3994575 sha256=195e412e9eb6c10e5d0813565e90302a319e88751c9e23539a252d78abf278c9
  Stored in directory: /root/.cache/pip/wheels/a9/63/f9/a653f9c21575e6ff271ee6a49939aa002005174cea6c35919d
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=fa3a210eb94ffa439edc479c7fa1145f3995a6b8696fe23ad49eb1cac0aeca7b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built seqeval fairscale jsonnet pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which i

In [4]:
import tensorflow_datasets as tfds
from tensorflow_datasets.text import Snli
import os

In [5]:
from __future__ import division

import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Sparsemax(nn.Module):
    """Sparsemax function."""

    def __init__(self, dim=None):
        """Initialize sparsemax activation
        
        Args:
            dim (int, optional): The dimension over which to apply the sparsemax function.
        """
        super(Sparsemax, self).__init__()

        self.dim = -1 if dim is None else dim

    def forward(self, input):
        """Forward function.
        Args:
            input (torch.Tensor): Input tensor. First dimension should be the batch size
        Returns:
            torch.Tensor: [batch_size x number_of_logits] Output tensor
        """
        # Sparsemax currently only handles 2-dim tensors,
        # so we reshape to a convenient shape and reshape back after sparsemax
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)

        # Translate input by max for numerical stability
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)

        # Sort input in descending order.
        # (NOTE: Can be replaced with linear time selection method described here:
        # http://stanford.edu/~jduchi/projects/DuchiShSiCh08.html)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, step=1, device=device, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        # Determine sparsity of projection
        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]

        # Compute threshold function
        zs_sparse = is_gt * zs

        # Compute taus
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)

        # Sparsemax
        self.output = torch.max(torch.zeros_like(input), input - taus)

        # Reshape back to original shape
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)

        return output

    def backward(self, grad_output):
        """Backward function."""
        dim = 1

        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))

        return self.grad_input

In [6]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [7]:
ac_dict = {"positive" : 1, "negative" : 0}

In [8]:
import os
from operator import itemgetter    
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')

# from keras import models, regularizers, layers, optimizers, losses, metrics
# from keras.models import Sequential
# from keras.layers import Dense
# from tensorflow.keras.utils import to_categorical
# from keras.layers import LayerNormalization
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [9]:
import torch

In [10]:
print(torch.cuda.is_available())

True


In [11]:
print(len(ac_dict))

2


In [12]:
model_version = 'allenai/longformer-base-4096'
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
import torch.nn as nn
import torch

In [14]:
from huggingface_hub import create_repo

In [15]:
from transformers import LongformerTokenizer, LongformerModel


In [16]:

from transformers import LongformerConfig, LongformerTokenizer, LongformerModel

tokenizer = LongformerTokenizer.from_pretrained(model_version,
                                          bos_token = "[CLS]",
                                          eos_token = "[SEP]")
transformer_model = LongformerModel.from_pretrained(model_version, attention_window = 64, output_attentions = True).to(device)
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)
cross_entropy_layer = nn.CrossEntropyLoss()
normalizing_layer = Sparsemax(dim = 1)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
print(transformer_model.config.hidden_size)

768


In [18]:
device

device(type='cuda', index=0)

In [19]:
from tqdm import tqdm

In [20]:
from typing import List, Tuple

In [21]:

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
num_words=10000)

17473536/17464789 [==============================] - 0s 0us/step


In [22]:
word_index = imdb.get_word_index()

reverse_word_index = dict(
[(value, key) for (key, value) in word_index.items()])

decoded_review = ' '.join(
[reverse_word_index.get(i - 3, '?') for i in train_data[10]])

print(decoded_review)

1654784/1641221 [==============================] - 0s 0us/step
? french horror cinema has seen something of a revival over the last couple of years with great films such as inside and ? romance ? on to the scene ? ? the revival just slightly but stands head and shoulders over most modern horror titles and is surely one of the best french horror films ever made ? was obviously shot on a low budget but this is made up for in far more ways than one by the originality of the film and this in turn is ? by the excellent writing and acting that ensure the film is a winner the plot focuses on two main ideas prison and black magic the central character is a man named ? sent to prison for fraud he is put in a cell with three others the quietly insane ? body building ? marcus and his retarded boyfriend daisy after a short while in the cell together they stumble upon a hiding place in the wall that contains an old ? after ? part of it they soon realise its magical powers and realise they may be ab

In [23]:
def pad_batch(elems: List[List[int]], pad_token_id: int) -> List[List[int]]:
    """Pads all lists in elems to the maximum list length of any list in 
    elems. Pads with pad_token_id.
    """
    max_len = max([len(elem) for elem in elems])
    return [elem+[pad_token_id]*(max_len-len(elem)) for elem in elems]

def load_dataset(tokenizer):
  (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
  word_index = imdb.get_word_index()

  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

  new_train_data = []
  for tokenized_text in tqdm(train_data):
    decoded_sent = " ".join([reverse_word_index.get(word - 3, "?") for word in tokenized_text])
    # print(decoded_sent)
    new_encoded_text = tokenizer.encode(decoded_sent)
    new_train_data.append(new_encoded_text)
  
  new_test_data = []
  for tokenized_text in tqdm(test_data):
    decoded_sent = " ".join([reverse_word_index.get(word - 3, "?") for word in tokenized_text])
    new_encoded_text = tokenizer.encode(decoded_sent)
    new_test_data.append(new_encoded_text)

  return (new_train_data, train_labels), (new_test_data, test_labels)

def generator(dataset_data, dataset_label, max_len = 512, batch_size = 16):
  i = 0
  tokenized_threads, labels = [], []
  while i<len(dataset_data):
    tokenized_threads.append(dataset_data[i][:max_len])
    labels.append(dataset_label[i])
    i += 1
        
    if i%batch_size==0:
      yield (pad_batch(tokenized_threads, tokenizer.pad_token_id), 
                  labels)
      tokenized_threads, labels = [], []

In [24]:
(train_dataset, train_labels), (test_dataset, test_labels) = load_dataset(tokenizer)

100%|██████████| 25000/25000 [00:51<00:00, 489.75it/s]


In [25]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters()),
                       lr = 2e-5)

In [26]:
train_loss_list = []
test_loss_list = []
stats_list = []

In [27]:
def train(dataset_data, dataset_label, batch_size):
    global values_weight, values_bias, train_loss_list;
    accumulate_over = 32
    
    optimizer.zero_grad()
    print("Training")
    for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label,  batch_size = batch_size)):
        
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        labels = torch.tensor(labels, device=device, dtype=torch.long)


        loss = compute((tokenized_threads, 
                        labels,), False)

        print("\rLoss: ", loss.item(), end = " ")
        train_loss_list.append(loss.item())
        loss.backward()

        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    print()
    optimizer.step()

In [28]:
from datasets import load_metric
metric1 = load_metric("accuracy")
metric3 = load_metric("precision", average = "micro")
metric2 = load_metric("f1", average = "micro")
metric4 = load_metric("recall", average = "micro")
metric5 = load_metric("precision", average = "macro")
metric6 = load_metric("f1", average = "macro")
metric7 = load_metric("recall", average = "macro")


In [29]:
def evaluate(dataset_data, dataset_label, BATCH_SIZE):
    global test_loss_list, stats_list;
    int_to_labels = {v:k for k, v in ac_dict.items()}
    print('Evaluation')
    
    with torch.no_grad():
        for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE)):
            # print(comp_type_labels)
            #Cast to PyTorch tensor
            tokenized_threads = torch.tensor(tokenized_threads, device=device)
            labels = torch.tensor(labels, device=device)

            preds = compute((tokenized_threads, 
                            labels,), pred=True)
            loss = compute((tokenized_threads, labels,), pred = False)
            
            metric1.add_batch(predictions=preds, 
                            references=labels,)
                            #tokenized_threads=tokenized_threads.cpu().tolist())
            metric2.add_batch(predictions=preds, 
                            references=labels,)
            metric3.add_batch(predictions=preds, 
                            references=labels,)
            metric4.add_batch(predictions=preds, 
                            references=labels,)
            metric5.add_batch(predictions=preds, 
                            references=labels,)
            metric6.add_batch(predictions=preds, 
                            references=labels,)
            metric7.add_batch(predictions=preds, 
                            references=labels,)
            
            print("\rLoss: ", loss.item(), end = " ")
            test_loss_list.append(loss.item())

    print()
    stats = []
    stats.append(metric1.compute())
    stats.append(metric5.compute(average = "macro"))
    stats.append(metric6.compute(average = "macro"))
    stats.append(metric7.compute(average = "macro"))
    print(stats[0])
    print("Micro")
    print(metric2.compute(average = "micro"))
    print(metric3.compute(average = "micro"))
    print(metric4.compute(average = "micro"))
    print("Macro")
    print(stats[1])
    print(stats[2])
    print(stats[3])
    stats_list.append(stats)

In [30]:

(train_dataset_data, train_dataset_label), (test_dataset_data, test_dataset_label) = load_dataset(tokenizer)

100%|██████████| 25000/25000 [00:49<00:00, 508.33it/s]


In [31]:
transformer_model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

In [32]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor], pred: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, labels = batch
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, torch.tensor(1).to(device), torch.tensor(0).to(device))
    x = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)
#     print(tokenized_threads.shape)
#     print(x[0].last_hidden_state, x[1].shape, len(x[2]))
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[0][:, 0, :])

    logits = normalizing_layer(logits)
    if(pred):
      return torch.argmax(logits, dim = 1)

    ce_loss = cross_entropy_layer(logits, labels)

    return ce_loss

In [33]:
import numpy as np
def shuffle(data, labels):
  idx = np.random.permutation(len(data))
  x,y = np.array(data)[idx], np.array(labels)[idx]
  return x.tolist(), y.tolist()

In [34]:
BATCH_SIZE = 4

In [35]:
print(tokenizer.decode([101]))

 like


In [36]:
print(device)

cuda:0


In [37]:
import pickle


In [38]:
linear_path = "Model_longformer/linear_layer_64.pt"
cross_path = "Model_longformer/cross_entropy_layer_64.pt"
tokenizer_path = "Model_longformer/tokenizer_pre_64.pkl"
transformer_path = "Model_longformer/transformer_layer_64.pt"
offset_path = "Model_longformer/offset_64.pkl"
data_path = "Model_longformer/data_64.pkl"

In [39]:
print(len(train_dataset_data))

25000


In [40]:
train_dataset_data, train_dataset_label = shuffle(train_dataset_data, train_dataset_label)
test_dataset_data, test_dataset_label = shuffle(test_dataset_data, test_dataset_label)
X_train, Y_train = train_dataset_data, train_dataset_label
X_test, Y_test = test_dataset_data, test_dataset_label

In [41]:
offset = 0
with open(offset_path, "wb") as f:
    pickle.dump(offset, f)
with open(offset_path, "rb") as f:
    offset = pickle.load(f)
torch.save(linear_layer.state_dict(), linear_path)
torch.save(cross_entropy_layer.state_dict(), cross_path)
torch.save(transformer_model.state_dict(), transformer_path)
with open(data_path, "wb") as f:
  pickle.dump([train_loss_list, test_loss_list, stats_list], f)
with open(tokenizer_path, "wb") as f:
  pickle.dump(tokenizer, f)

In [42]:

print(max([len(i) for i in train_dataset_data]))
  
print(len(train_dataset_data[10]))

2675
262


In [43]:
n_epochs = 1

for epoch in range(n_epochs):
    train_dataset_data, train_dataset_label = shuffle(train_dataset_data, train_dataset_label)
    test_dataset_data, test_dataset_label = shuffle(test_dataset_data, test_dataset_label)
    X_train, Y_train = train_dataset_data, train_dataset_label
    X_test, Y_test = test_dataset_data, test_dataset_label
    print(f"------------EPOCH {epoch+1}---------------")
    #loading data
    linear_layer.load_state_dict(torch.load(linear_path, map_location=device))
    cross_entropy_layer.load_state_dict(torch.load(cross_path, map_location=device))
    transformer_model.load_state_dict(torch.load(transformer_path , map_location=device))
    with open(tokenizer_path, "rb") as f:
      tokenizer = pickle.load(f)
    with open(data_path, "rb") as f:
      data = pickle.load(f)
      stats_list = data[2]
      train_loss_list = data[0]
      test_loss_list = data[1]

    #training and evaluating 
    print("Train")
    train(X_train, Y_train, BATCH_SIZE)
    torch.save(linear_layer.state_dict(), linear_path)
    torch.save(cross_entropy_layer.state_dict(), cross_path)
    torch.save(transformer_model.state_dict(), transformer_path)
    
    with open(data_path, "wb") as f:
      pickle.dump([train_loss_list, test_loss_list, stats_list], f)
    with open(tokenizer_path, "wb") as f:
      pickle.dump(tokenizer, f)
      
    print("Test accuracy")
    evaluate(X_test, Y_test, BATCH_SIZE)


------------EPOCH 1---------------
Train
Training
Loss:  0.3418155908584595 
Test accuracy
Evaluation
Loss:  0.5632617473602295 
{'accuracy': 0.8764}
Micro
{'f1': 0.8764}
{'precision': 0.8764}
{'recall': 0.8764}
Macro
{'precision': 0.8765365026153865}
{'f1': 0.8763887970672337}
{'recall': 0.8764000000000001}


In [44]:
print(offset)

0


In [45]:
torch.save(linear_layer.state_dict(), linear_path)
torch.save(cross_entropy_layer.state_dict(), cross_path)
torch.save(transformer_model.state_dict(), transformer_path)

with open(data_path, "wb") as f:
  pickle.dump([train_loss_list, test_loss_list, stats_list], f)
with open(tokenizer_path, "wb") as f:
  pickle.dump(tokenizer, f)

In [46]:
linear_layer.load_state_dict(torch.load(linear_path, map_location=device))
cross_entropy_layer.load_state_dict(torch.load(cross_path, map_location=device))
transformer_model.load_state_dict(torch.load(transformer_path , map_location=device))
with open(data_path, "rb") as f:
  data = pickle.load(f)
  stats_list = data[2]
  train_loss_list = data[0]
  test_loss_list = data[1]
with open(tokenizer_path, "rb") as f:
  tokenizer = pickle.load(f)

In [47]:
from multiprocessing import Process, Manager, Array
manager = Manager()

In [48]:
graph_attention = []
for i in range(12):
  graph_attention.append({})
mapping_ind = {}
rev_mapping_ind = {}

In [49]:
cnt = 0
threshold = 0.01

In [50]:
import multiprocessing as mp
from tqdm import tqdm

In [51]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [52]:


my_arr = [1 for i in range(100)]

def func(i, j , k):
    global my_arr
    my_arr[i] = 3
    

indexes = [(i, i, i) for i in range(12)]
pool = mp.Pool(min(12, mp.cpu_count()))
pool.starmap(func, indexes)
pool.close()

In [53]:
type(mp.cpu_count())

int

In [54]:
temp_attention = []
def init_arr(arr):
    globals()['arr'] = arr
from copy import deepcopy
def store_attention(lay, j, tokens):
        
    global mapping_ind, graph_attention, temp_attention;
    maximum_attention = np.max(temp_attention)
    weights = temp_attention[j, lay, :, :]
    temp_dict = {}
    for ind_1 in range(len(tokens)):
        for ind_2 in range(len(tokens)):
            if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                continue
            node_1 = mapping_ind[tokens[ind_1]]
            node_2 = mapping_ind[tokens[ind_2]]
            weight = weights[ind_1, ind_2]

            if(weight < threshold*maximum_attention):
                continue
            if(node_1 not in temp_dict):
                temp_dict[node_1] = {}
            
            if(node_2  not in temp_dict[node_1]):
                temp_dict[node_1][node_2] = (weight, 1)
            else:
                temp_dict[node_1][node_2] = (weight*temp_dict[node_1][node_2][1]/(temp_dict[node_1][node_2][1] + 1), temp_dict[node_1][node_2][1] + 1)
    graph_attention[lay] = deepcopy(temp_dict)
#     print(graph_attention)
def attention_graph(dataset_data, dataset_label, BATCH_SIZE):
    
    
    accumulate_over = 4
    global cnt, temp_attention, manager;
    optimizer.zero_grad()

    for i, (tokenized_threads, labels) in tqdm(enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE))):
    
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        temp_attention = attention[:, :, :, :]
        maximum_attention = np.max(attention)
        for j, tokenized_thread in enumerate(tokenized_threads):
            tokens = tokenizer.convert_ids_to_tokens(tokenized_thread) 
            # print(len(tokens))
            # print(tokenized_thread.shape)
            for tok in tokens:
                if(tok not in mapping_ind):
                    mapping_ind[tok] = cnt;
                    cnt += 1
            
    for i, (tokenized_threads, labels) in tqdm(enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE))):
    
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        temp_attention = attention[:, :, :, :]
        maximum_attention = np.max(attention)
        for j, tokenized_thread in enumerate(tokenized_threads):
            tokens = tokenizer.convert_ids_to_tokens(tokenized_thread) 

#             params = [(i1, j, tokens) for i1 in range(12)]
#             pool = mp.Pool(min(12, mp.cpu_count()))
#             pool.starmap(store_attention, params)
#             pool.close()
            for lay in range(12):
                weights = attention[j, lay, :, :]
                for ind_1 in range(len(tokens)):
                    for ind_2 in range(len(tokens)):
                        if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                            continue

                        node_1 = mapping_ind[tokens[ind_1]]
                        node_2 = mapping_ind[tokens[ind_2]]
                        weight = weights[ind_1, ind_2]
                        if(weight < threshold*maximum_attention):
                            continue
                        if(node_1 not in graph_attention[lay]):
                            graph_attention[lay][node_1] = {}
                        if(node_2  not in graph_attention[lay][node_1]):
                            graph_attention[lay][node_1][node_2] = (weight, 1)
                        else:
                            graph_attention[lay][node_1][node_2] = ((weight + (graph_attention[lay][node_1][node_2][0])*graph_attention[lay][node_1][node_2][1])/(graph_attention[lay][node_1][node_2][1] + 1), graph_attention[lay][node_1][node_2][1] + 1)
#         print(cnt, cnt**2)              


In [55]:
def extract_data(dataset_data, dataset_label, BATCH_SIZE):
    example = [[], [], []]
    for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE)):
        for j in range(BATCH_SIZE):
            if(len(example[0]) < 5 or len(example[1]) < 5 or len(example[2]) < 5):
                example[labels[j]].append((tokenizer.convert_ids_to_tokens(tokenized_threads[j]), tokenized_threads[j]))
            else:
                break
    return example

In [56]:
text = extract_data(test_dataset_data, test_dataset_label, BATCH_SIZE)

In [57]:
import random

In [58]:
def find_degree(nodes, graph):
    out_degree = {}
    in_degree = {}
    for node_1 in graph:
        for node_2 in graph[node_1]:
            if(node_1 not in out_degree):
                out_degree[node_1] = 0
            if(node_2 not in in_degree):
                in_degree[node_2] = 0;
            in_degree[node_2] += 1;
            out_degree[node_1] += 1;
    ret_out = [out_degree[i] for i in nodes]
    ret_in = [in_degree[i] for i in nodes]
    return ret_out, ret_in
def find_topk_tokens(graph, k):
    attention = {}
    attention_inward = {}
    for node_1 in graph:
        if(node_1 not in attention):
            attention[node_1] = (0, 0)
        for node_2 in graph[node_1]:
            
            if(node_2 not in attention_inward):
                attention_inward[node_2] = (0, 0)
            attention[node_1] = ((graph[node_1][node_2][0] + (attention[node_1][0])*attention[node_1][1])/(attention[node_1][1] + 1), attention[node_1][1] + 1)
            attention_inward[node_2] = ((graph[node_1][node_2][0] + (attention_inward[node_2][0])*attention_inward[node_2][1])/(attention_inward[node_2][1] + 1), attention_inward[node_2][1] + 1)
    out_max = [i[0] for i in find_maxk(attention, k)]
    in_max = [i[0] for i in find_maxk(attention_inward, k)]
    return in_max, out_max, [i[1] for i in find_maxk(attention_inward, k)], [i[1] for i in find_maxk(attention, k)], find_mink(attention, k)[0][1], find_mink(attention_inward, k)[0][1]
def find_mink(attention, k):
    ret = []
    for i in attention:
        if(i == mapping_ind["[CLS]"] or i == mapping_ind["[SEP]"]):
            continue
        ret.append((i, attention[i][0]))
    ret.sort(key = lambda x: x[1])
    return ret[:k]
def find_maxk(attention, k):
    ret = []
    for i in attention:
        if(i == mapping_ind["[CLS]"] or i == mapping_ind["[SEP]"]):
            continue
        ret.append((i, attention[i][0]))
    ret.sort(key = lambda x: x[1])
    ret.reverse()
    # print(ret)
    return ret[:k]

In [59]:
def remove_pad(given_s):
    l = given_s.split()
    ret = []
    for i in l:
        if(i != "[PAD]"):
            ret.append(i)
    return " ".join(ret)

In [60]:
def replacing(tok_mapping, tokenized_thread, tokens):
    ret_tokenized_thread = tokenized_thread.clone()
    ind = []
    for i in range(len(tokens)):
        if(mapping_ind[tokens[i]] in tok_mapping):
            ret_tokenized_thread[0, i] = tokenizer.encode(tok_mapping[mapping_ind[tokens[i]]])[1];
            ind.append(i)
    return ret_tokenized_thread, ind

In [61]:
vocab_dict = tokenizer.get_vocab()

In [62]:

vocab = []
for i in vocab_dict:
  if(i not in ["[CLS]", '[PAD]', '[SEP]']):
    vocab.append(i)



In [63]:
import random
def create_mapping(tok, ind):
    ret_dict = {}
    for k, i in enumerate(tok):
        if((k + i) < len(vocab)):
            ret_dict[i] = vocab[k + ind]
        else:
            ret_dict[i] = random.choice(vocab)
        ret_dict[i] = random.choice(vocab)
    return ret_dict

In [64]:
def get_embedding(ind, tokenized):
    pad_mask = torch.where(tokenized!=tokenizer.pad_token_id, 1, 0)
    embedding = transformer_model(input_ids=tokenized,attention_mask=pad_mask,).last_hidden_state
    ret_embedding = []
    for i in ind:
        ret_embedding.append(embedding[:, i, :].reshape((embedding.shape[2])))
    return ret_embedding

In [65]:
replacement = 100
trials = 3
trial_candidate = 5
select_max = 5

In [66]:
print(len(text[1]))

12500


In [67]:
import matplotlib.pyplot as plt
from scipy import spatial
text[0] = text[0][:3]
text[1] = text[1][:3]
text[2] = text[2][:3]

In [68]:
def predict_proba(tokenized_threads):
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,attention_mask=pad_mask,).last_hidden_state[:, 0, :])
    logits = torch.exp(logits)
    z = torch.sum(logits)
    logits = torch.divide(logits, z)
    return logits
def mask_out_top5(text_lab, label, label_ind, flag):
    accumulate_over = 4
    global cnt, temp_attention, manager;
    optimizer.zero_grad()

    for i1, (tokens, tokenized_threads) in enumerate(text_lab):
        print("\n\n\n")
        print("###### Example :", str(i1), "label", label, "########")
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device).reshape((1, len(tokenized_threads)))
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
        
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        graph = dict()
        for tok in tokens:
            if(tok not in mapping_ind):
                mapping_ind[tok] = cnt;
                cnt += 1

        for lay in range(12):
            weights = attention[0, lay, :, :]

            for ind_1 in range(len(tokens)):
                for ind_2 in range(len(tokens)):
                    if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                        continue
                    node_1 = mapping_ind[tokens[ind_1]]
                    node_2 = mapping_ind[tokens[ind_2]]
                    rev_mapping_ind[node_1] = tokens[ind_1]
                    rev_mapping_ind[node_2] = tokens[ind_2]
                    weight = weights[ind_1, ind_2]
                    if(node_1 not in graph):
                        graph[node_1] = {}
                    if(node_2  not in graph[node_1]):
                        graph[node_1][node_2] = (weight, 1)
                    else:
                        graph[node_1][node_2] = ((weight + (graph[node_1][node_2][0])*graph[node_1][node_2][1])/(graph[node_1][node_2][1] + 1), graph[node_1][node_2][1] + 1)

        topk_in, topk_out, topk_in_attention, topk_out_attention, topk_in_min, topk_out_min  = find_topk_tokens(graph, select_max)

        
        candidate = random.sample(topk_in, trial_candidate)
        for j in range(trials):
            random.shuffle(vocab)
            print([ rev_mapping_ind[z] for z in candidate], "\n")
            in_logits_l2_norm = []
            for l in tqdm(range(0, min(len(vocab), 1000), select_max)):
                top5_in_mapping = create_mapping(candidate, l)

                example_in, ind_in = replacing(top5_in_mapping, tokenized_threads, tokens)

                actual_in = get_embedding(ind_in, tokenized_threads)
                replaced_in = get_embedding(ind_in, example_in)

                if(len(ind_in) >= 1):
                    final_actual_in = torch.divide(sum(actual_in),len(actual_in)).cpu().detach().numpy()
                    final_replaced_in = torch.divide(sum(replaced_in) , len(replaced_in)).cpu().detach().numpy()
                else:
                    continue
                if(predict_proba(example_in).cpu().detach().numpy().tolist()[0][label_ind] < 0.5):
                    continue
                subtract_in = np.subtract(final_actual_in, final_replaced_in)
                if(flag == "L2"):
                    in_logits_l2_norm.append((np.linalg.norm(subtract_in), predict_proba(example_in).cpu().detach().numpy().tolist()[0][label_ind]))
                elif(flag == "Cosine"):
                    in_logits_l2_norm.append((spatial.distance.cosine(final_actual_in, final_replaced_in), predict_proba(example_in).cpu().detach().numpy().tolist()[0][label_ind]))
                    
                    
            in_logits_l2_norm.sort(key = lambda x : x[0])
            

            plt.scatter([i[0] for i in in_logits_l2_norm],[i[1] for i in in_logits_l2_norm])
        print("| Min:", topk_in_min)
        for i in range(len(topk_in)):
            print("Token:", rev_mapping_ind[topk_in[i]], "| attention:", topk_in_attention[i])
        plt.title('Example ' + str(i) + ' in')
        plt.xlabel(flag + ' norm')
        plt.ylabel('probability of '+ label )
        plt.show()
        plt.clf()
        
        candidate = random.sample(topk_out, trial_candidate)
        for j in range(trials):

            random.shuffle(vocab)
            print([ rev_mapping_ind[z] for z in candidate], "\n")
            out_logits_l2_norm = []
            for l in tqdm(range(0, min(len(vocab), 1000), select_max)):
                top5_out_mapping = create_mapping([j], l)

                
                
                example_out, ind_out = replacing(top5_out_mapping, tokenized_threads, tokens)

                actual_out = get_embedding(ind_out, tokenized_threads)
                replaced_out = get_embedding(ind_out, example_out)
#                 print(sum(actual_out))
                if(len(ind_out) >= 1):
                    final_actual_out = torch.divide(sum(actual_out) , len(actual_out)).cpu().detach().numpy()
                    final_replaced_out = torch.divide(sum(replaced_out) , len(replaced_out)).cpu().detach().numpy()
                else:
                    continue
                if(predict_proba(example_out).cpu().detach().numpy().tolist()[0][label_ind] < 0.50):
                    continue    
                subtract_out = np.subtract(final_actual_out, final_replaced_out)
                if(flag == "L2"):
                    out_logits_l2_norm.append((np.linalg.norm(subtract_out), predict_proba(example_out).cpu().detach().numpy().tolist()[0][label_ind]))
                elif(flag == "Cosine"):
                    out_logits_l2_norm.append((spatial.distance.cosine(final_actual_out, final_replaced_out), predict_proba(example_out).cpu().detach().numpy().tolist()[0][label_ind]))
                    
            out_logits_l2_norm.sort(key = lambda x : x[0])
            
            plt.scatter([i[0] for i in out_logits_l2_norm],[i[1] for i in out_logits_l2_norm])

        print("| Min:", topk_out_min)
        for i in range(len(topk_out)):
            print("Token:", rev_mapping_ind[topk_out[i]], "| attention:", topk_out_attention[i])
        plt.title('Example ' + str(i) + ' out')
        plt.xlabel(flag + ' norm')
        plt.ylabel('probability of ' + label )
        plt.show()
        plt.clf()

In [69]:

mask_out_top5(text[0], "neutral", 0, "L2")





###### Example : 0 label neutral ########


IndexError: ignored

In [ ]:
mask_out_top5(text[1], "entailment", 1, "L2",)

In [ ]:
mask_out_top5(text[2], "contradiction", 2, "L2",)

In [ ]:
mask_out_top5(text[0], "neutral", 0, "Cosine")

In [ ]:
mask_out_top5(text[1], "entailment", 1, "Cosine",)

In [ ]:
mask_out_top5(text[2], "contradiction", 2, "Cosine",)

In [ ]:
select_max_imp = 100

In [ ]:
def calculate_imp(true_logits, true_class, perturbed_logits, perturbed_class):
  if(perturbed_class == true_class):
    return true_logits[true_class] - perturbed_logits[true_class]
  return true_logits[true_class] - perturbed_logits[true_class] + true_logits[perturbed_class] - perturbed_logits[perturbed_class]

In [ ]:
print(tokenizer.decode([1532]))

In [ ]:
def intersection(a, b):
  cnt = 0;
  print(a)
  print(b)
  for i in range(min(len(a), len(b))):
      if(a[i] == b[i]):
          cnt += 1;
      
  return (cnt/min(len(a), len(b)))*100

In [ ]:
def find_intersection_top5(text_lab, label, label_ind, flag):
    accumulate_over = 4
    global cnt, temp_attention, manager;
    optimizer.zero_grad()

    for i, (tokens, tokenized_threads) in enumerate(text_lab):
        print("\n\n\n")
        print("###### Example :", str(i), "label", label, "########")
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device).reshape((1, len(tokenized_threads)))
        
        pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
        
        attention = transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,)[-1][0].cpu().detach().numpy()
        graph = dict()
        
        for tok in tokens:
            if(tok not in mapping_ind):
                mapping_ind[tok] = cnt;
                cnt += 1

        for lay in range(12):
            weights = attention[0, lay, :, :]
            for ind_1 in range(len(tokens)):
                for ind_2 in range(len(tokens)):
                    if(tokens[ind_1] in ["[NEWLINE]", '[PAD]'] or tokens[ind_2] in ["[NEWLINE]", '[PAD]']):
                        continue
                    node_1 = mapping_ind[tokens[ind_1]]
                    node_2 = mapping_ind[tokens[ind_2]]
                    rev_mapping_ind[node_1] = tokens[ind_1]
                    rev_mapping_ind[node_2] = tokens[ind_2]
                    weight = weights[ind_1, ind_2]
                    if(node_1 not in graph):
                        graph[node_1] = {}
                    if(node_2  not in graph[node_1]):
                        graph[node_1][node_2] = (weight, 1)
                    else:
                        graph[node_1][node_2] = ((weight + (graph[node_1][node_2][0])*graph[node_1][node_2][1])/(graph[node_1][node_2][1] + 1), graph[node_1][node_2][1] + 1)
        logits = predict_proba(tokenized_threads)
        class_assigned = torch.argmax(logits[0, :])
        topk_in, topk_out, topk_in_attention, topk_out_attention, topk_in_min, topk_out_min = find_topk_tokens(graph, select_max_imp)
        arr = tokenized_threads.cpu().detach().clone().numpy()

        candidates = []
        for j in range(len(tokens)):
          temp_tokenized_threads = np.concatenate([arr[0, : j], arr[0, j + 1:], np.array([tokenizer.pad_token_id])])
    
          temp_logits = predict_proba(torch.tensor(temp_tokenized_threads, device=device).reshape((1, len(temp_tokenized_threads))))

          temp_class_assigned = torch.argmax(temp_logits)

          importance_score = calculate_imp(logits[0], class_assigned, temp_logits[0], temp_class_assigned)
          candidates.append((arr[0, j], importance_score))
        candidates.sort(key = lambda x : x[1])
        candidates.reverse()
        print(candidates)
        print("intersection between in : ", intersection([tokenizer.decode(i[0]) for i in candidates[:select_max_imp] if tokenizer.decode(i[0]) not in ["[CLS]", "[PAD]", "[SEP]"]], [rev_mapping_ind[z] for z in topk_in]))
        print("intersection between out : ",intersection([tokenizer.decode(i[0]) for i in candidates[:select_max_imp] if tokenizer.decode(i[0]) not in ["[CLS]", "[PAD]","[SEP]"]], [rev_mapping_ind[z] for z in topk_out]))

In [ ]:

find_intersection_top5(text[2], "contradiction", 2, "Cosine",)

In [ ]:
find_intersection_top5(text[1], "entailment", 1, "L2",)

In [ ]:
find_intersection_top5(text[0], "neutral", 0, "L2")